<a href="https://colab.research.google.com/github/JV-Machado/Chess_Classification/blob/master/Chess_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/TCC/Chess-image-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/TCC/Folder_Chess_Dataset')

**Organização dos dados**

In [ ]:
import glob
from shutil import copyfile
import os

imgs = glob.glob("/content/drive/MyDrive/TCC/Folder_Chess_Dataset/Chess-image-dataset/*/*")
for img in imgs:
  class_name = img.split("/")[-2][0:]
  img_name = img.split("/")[-1][0:]
  copyfile(img, f'/content/drive/MyDrive/TCC/data/-{class_name}-{img_name}')

In [ ]:
import pandas as pd
import glob

imgs_path = glob.glob('/content/drive/MyDrive/TCC/data/*')
lista = []

for img_path in imgs_path:
  name = img_path.split('-')[-2][0:]
  # img = img_path.split('-')[-1][0:]
  lista.append([img_path, name])

In [ ]:
df = pd.DataFrame(columns=["Image_Path", "Image_Class"], data=lista)
df

In [ ]:
categories = pd.factorize(df['Image_Class'])[1]
print(categories)
df['Image_Class'] = pd.factorize(df['Image_Class'])[0]
df

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow
import torch
from PIL import Image
from torch.utils.data import DataLoader,Dataset
import os

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
import os

preprocess = transforms.Compose([
    transforms.Resize((100,100)),
    transforms.ToTensor(),
    transforms.Normalize(
        (0.6741, 0.6644, 0.6534), 
        (0.3837, 0.3853, 0.3900))
])

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, df, transform = None):
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        image = Image.fromarray(cv2.cvtColor(cv2.imread(self.df.iloc[index,0]),cv2.COLOR_BGR2RGB))
        image=self.transform(image)
        y = torch.tensor(int(self.df.iloc[index,1]))

        return image, y

In [ ]:
train_dataset = ImageDataset(df, transform = preprocess)

In [ ]:
# imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
# imgs.shape

In [ ]:
# imgs.view(3, -1).mean(dim=1)

In [ ]:
# imgs.view(3, -1).std(dim=1)

In [ ]:
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split

In [ ]:
validation_data_size = round(0.20*(len(train_dataset)))
# test_data_size = round(0.2*(len(train_dataset)))
train_data_size = len(train_dataset) - validation_data_size 

train_data, validation_data  = random_split(train_dataset, [train_data_size, validation_data_size])
print(f"Length of Train Data : {len(train_data)}")
# print(f"Length of Test Data : {len(test_data)}")
print(f"Length of Validation Data : {len(validation_data)}")

Length of Train Data : 521
Length of Validation Data : 130


In [ ]:
batch_size = 16

train_DL = DataLoader(train_data, batch_size, shuffle = True)
# test_DL = DataLoader(test_data, batch_size, shuffle = True)
validation_DL = DataLoader(validation_data, batch_size, shuffle = True)

**Treinamento com CNN**

In [ ]:
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
def train_loop(model, dataloader, loss_func, optimizer):
    model.train()
    cumloss = 0.0
    running_accuracy = 0.0

    for i, data in enumerate(dataloader):
        imgs, labels = data
    
        imgs, labels = imgs.to(device), labels.to(device)
    
        optimizer.zero_grad()

        pred = model(imgs)

        loss = loss_func(pred, labels)
        loss.backward()
        optimizer.step()

        cumloss += loss.item()
        
        ps = F.softmax(pred,-1)
        top_p, top_class = ps.topk(k = 1, dim = 1)
        equals = top_class == labels.view(*top_class.shape)

        accuracy = torch.mean(equals.type(torch.float))

        running_accuracy += accuracy

    return cumloss / len(dataloader), running_accuracy/len(dataloader)

def validate(model, dataloader, loss_func):
    model.eval()
    cumloss = 0.0
    running_accuracy_val = 0.0
    with torch.no_grad():
        for imgs, labels in dataloader:
            imgs, labels = imgs.to(device), labels.to(device)

            pred = model(imgs)
            loss = loss_func(pred, labels)
            cumloss += loss.item()

            ps = F.softmax(pred,-1)
            top_p, top_class = ps.topk(k = 1, dim = 1)
            equals = top_class == labels.view(*top_class.shape)

            accuracy_val = torch.mean(equals.type(torch.float))

            running_accuracy_val += accuracy_val

        return cumloss / len(dataloader), running_accuracy_val/len(dataloader)

In [ ]:
model_ft = models.vgg16(pretrained=True)
model_ft = model_ft.to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_ft.parameters(), lr=0.001)

In [ ]:
num_features = models.vgg16().classifier[6].in_features
model_ft.fc = nn.Linear(num_features, 5)

In [ ]:
epochs = 25
conv_train_losses = []
conv_test_losses = []
for t in range(epochs):
    train_loss, acc_train = train_loop(model_ft, train_DL, loss_func, optimizer)
    conv_train_losses.append(train_loss)
    print(f"\rEpoch: {t}; Train Loss: {train_loss} Accuracy train: {acc_train}")
    test_loss, acc_test = validate(model_ft, validation_DL, loss_func)
    conv_test_losses.append(test_loss)
    print(f"\rEpoch: {t}; Validate Loss: {test_loss} Accuracy validation: {acc_test}")

**Matriz de Confusão**

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd

y_pred = []
y_true = []
nb_classes = 5

confusion_matrix = torch.zeros(nb_classes, nb_classes)

for imgs, labels in validation_DL:
  imgs, labels = imgs.to(device), labels.to(device)

  pred = model_ft(imgs)

  _, preds = torch.max(pred, 1)
  for t, p in zip(labels.view(-1), preds.view(-1)):
    confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

In [ ]:
class_names = list(categories)
df_cm = pd.DataFrame(confusion_matrix, index=class_names, columns=class_names).astype(int)
heatmap = sns.heatmap(df_cm, annot=True, fmt="d")

heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right',fontsize=15)
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right',fontsize=15)
plt.ylabel('True label')
plt.xlabel('Predicted label')